In [1]:
import sys
dataDir = '../../VQA'
sys.path.insert(0, '%s/PythonHelperTools/vqaTools' %(dataDir))

In [2]:
from vqa import VQA
from vqaEvaluation.vqaEval import VQAEval
import matplotlib.pyplot as plt
import skimage.io as io
import json
import random
import os

In [3]:
# set up file names and paths
versionType ='v2_' # this should be '' when using VQA v2.0 dataset
taskType    ='OpenEnded' # 'OpenEnded' only for v2.0. 'OpenEnded' or 'MultipleChoice' for v1.0
dataType    ='mscoco'  # 'mscoco' only for v1.0. 'mscoco' for real and 'abstract_v002' for abstract for v1.0. 
dataSubType ='train2014'
annFile     ='%s/Annotations/%s%s_%s_annotations.json'%(dataDir, versionType, dataType, dataSubType)
quesFile    ='%s/Questions/%s%s_%s_%s_questions.json'%(dataDir, versionType, taskType, dataType, dataSubType)
imgDir      ='%s/Images/%s/%s/' %(dataDir, dataType, dataSubType)
resultType  ='fake'
fileTypes   = ['results', 'accuracy', 'evalQA', 'evalQuesType', 'evalAnsType'] 

In [4]:
[resFile, accuracyFile, evalQAFile, evalQuesTypeFile, evalAnsTypeFile] = ['%s/Results/%s%s_%s_%s_%s_%s.json'%(dataDir, versionType, taskType, dataType, dataSubType, \
resultType, fileType) for fileType in fileTypes]  

In [5]:
vqa = VQA(annFile, quesFile)
vqaRes = vqa.loadRes(resFile, quesFile)

loading VQA annotations and questions into memory...
0:00:11.363723
creating index...
index created!
Loading and preparing results...     
DONE (t=0.92s)
creating index...
index created!


In [6]:
vqaEval = VQAEval(vqa, vqaRes, n=2)

In [7]:
"""
If you have a list of question ids on which you would like to evaluate your results, pass it as a list to below function
By default it uses all the question ids in annotation file
"""
vqaEval.evaluate() 

computing accuracy
Finshed Percent: [####################] 99% Done computing accuracy


In [8]:
# print accuracies
print ("\n")
print ("Overall Accuracy is: %.02f\n" %(vqaEval.accuracy['overall']))
print ("Per Question Type Accuracy is the following:")
for quesType in vqaEval.accuracy['perQuestionType']:
	print ("%s : %.02f" %(quesType, vqaEval.accuracy['perQuestionType'][quesType]))
print ("\n")
print ("Per Answer Type Accuracy is the following:")
for ansType in vqaEval.accuracy['perAnswerType']:
	print ("%s : %.02f" %(ansType, vqaEval.accuracy['perAnswerType'][ansType]))
print ("\n")



Overall Accuracy is: 84.89

Per Question Type Accuracy is the following:
what is this : 85.84
what : 77.08
what color is the : 92.01
is this : 98.35
what is the person : 87.22
what is in the : 81.36
is the : 96.35
is there : 98.02
what is the : 79.09
is : 96.96
is the man : 96.88
none of the above : 82.32
does the : 96.26
are the : 95.76
was : 98.19
how many : 71.74
what color is : 93.41
what is the man : 84.45
what is on the : 76.65
what is : 78.96
how many people are in : 68.49
is there a : 97.93
is it : 98.96
is the person : 95.90
what does the : 69.48
is this a : 98.88
how : 53.49
are there : 97.92
has : 96.58
could : 97.73
where is the : 52.12
are : 93.86
are they : 98.09
are these : 98.18
why is the : 34.97
how many people are : 69.72
what color are the : 90.08
what type of : 80.08
where are the : 52.71
what kind of : 79.65
which : 67.98
do : 94.74
what is the name : 70.54
is this an : 99.32
what is the color of the : 92.44
what color : 90.67
what sport is : 98.11
what are : 84

In [10]:
# demo how to use evalQA to retrieve low score result
evals = [quesId for quesId in vqaEval.evalQA if vqaEval.evalQA[quesId]<35]   #35 is per question percentage accuracy
if len(evals) > 0:
	print ('ground truth answers')
	randomEval = random.choice(evals)
	randomAnn = vqa.loadQA(randomEval)
	vqa.showQA(randomAnn)

	print ('\n')
	print ('generated answer (accuracy %.02f)'%(vqaEval.evalQA[randomEval]))
	ann = vqaRes.loadQA(randomEval)[0]
	print ("Answer:   %s\n" %(ann['answer']))

	imgId = randomAnn[0]['image_id']
	imgFilename = 'COCO_' + dataSubType + '_'+ str(imgId).zfill(12) + '.jpg'
	if os.path.isfile(imgDir + imgFilename):
		I = io.imread(imgDir + imgFilename)
		plt.imshow(I)
		plt.axis('off')
		plt.show()

# save evaluation results to ./Results folder
json.dump(vqaEval.accuracy,     open(accuracyFile,     'w'))
json.dump(vqaEval.evalQA,       open(evalQAFile,       'w'))
json.dump(vqaEval.evalQuesType, open(evalQuesTypeFile, 'w'))
json.dump(vqaEval.evalAnsType,  open(evalAnsTypeFile,  'w'))

ground truth answers
Question: What color shirt is the woman wearing?
Answer 1: gray
Answer 2: gray
Answer 3: grey shirt
Answer 4: gray
Answer 5: gray
Answer 6: gray
Answer 7: gray
Answer 8: gray
Answer 9: blue
Answer 10: gray


generated answer (accuracy 30.00)
Answer:   blue

